In [1]:
import click
import json
import numpy as np
import pyvista as pv
import scipy.spatial as ss
from pymeshfix import MeshFix
import pandas as pd
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import MultiPolygon, Polygon

In [2]:
filename = "/path/to/cityjson/cityjson.json"

with open(filename) as file:
    cm = json.load(file)

if "transform" in cm:
    s = cm["transform"]["scale"]
    t = cm["transform"]["translate"]
    verts = [[v[0] * s[0] + t[0], v[1] * s[1] + t[1], v[2] * s[2] + t[2]]
            for v in cm["vertices"]]
else:
    verts = cm["vertices"]

# mesh points
vertices = np.array(verts)

In [3]:
surface_types = ["GroundSurface", "WallSurface", "RoofSurface"]

def get_surface_boundaries(geom):
    """Returns the boundaries for all surfaces"""

    if geom["type"] == "MultiSurface":
        return geom["boundaries"]
    elif geom["type"] == "Solid":
        return geom["boundaries"][0]
    else:
        raise Exception("Geometry not supported")

def to_polydata(geom, vertices):
    """Returns the polydata mesh from a CityJSON geometry"""

    boundaries = get_surface_boundaries(geom)

    f = [[len(r[0])] + r[0] for r in [f for f in boundaries]]
    faces = np.hstack(f)

    mesh = pv.PolyData(vertices, faces, n_faces=len(boundaries))

    if "semantics" in geom:        
        semantics = geom["semantics"]
        if geom["type"] == "MultiSurface":
            values = semantics["values"]
        else:
            values = semantics["values"][0]
        
        mesh["semantics"] = [semantics["surfaces"][i]["type"] for i in values]
    
    return mesh

In [4]:
obj = "GUID_16E64847-3529-4640-9C4A-8FF9EEBA3F0F_5"

building = cm["CityObjects"][obj]

In [5]:
dataset = to_polydata(building["geometry"][0], vertices)
dataset

PolyData (0x132174940)
  N Cells:	8
  N Points:	22997
  X Bounds:	7.825e+04, 7.904e+04
  Y Bounds:	4.576e+05, 4.583e+05
  Z Bounds:	2.463e+00, 3.748e+01
  N Arrays:	1

In [6]:
trimesh = dataset.triangulate()
trimesh.plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [7]:
trimesh.subdivide(1).plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [8]:
clean = trimesh.clean()
voxel = pv.voxelize(clean, density=clean.length/100)
voxel.plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [9]:
print(f"Voxel: {voxel.volume}")
print(f"Actual: {trimesh.volume}")

Voxel: 11901.150569005207
Actual: 11443.31443965503


In [10]:
p = pv.Plotter()

p.add_mesh(voxel, opacity=0.2)
p.add_mesh(pv.PolyData(np.mean(voxel.cell_centers().points, axis=0)), color='white')

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [11]:
from helpers.minimumBoundingBox import MinimumBoundingBox

In [12]:
obb_2d = MinimumBoundingBox([(p[0], p[1]) for p in dataset.clean().points])
obb_2d.area

458.2351560009694

In [13]:
obb_2d.area

458.2351560009694

In [14]:
ground_z = np.min(dataset.clean().points[:, 2])
height = np.max(dataset.clean().points[:, 2]) - ground_z
box = np.array([[p[0], p[1], ground_z] for p in list(obb_2d.corner_points)])

In [15]:
obb = pv.PolyData(box).delaunay_2d().extrude([0.0, 0.0, height])

In [16]:
p = pv.Plotter()

p.add_mesh(obb, opacity=0.3)
p.add_mesh(dataset.triangulate())

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [17]:
dataset.clean().triangulate().volume

11443.31443965503

In [18]:
m = MeshFix(obb.clean().triangulate())
m.repair()
fixed_obb = m.mesh